In [1]:
import pandas as pd
import copy
import random
import sys

In [2]:
# setting path
sys.path.append('../code')

In [3]:
import load_data as ld

In [4]:
# constsant
input_path = "../../dataset/"
output_path = "../data/"
ingr_percent = 1
good_rat = 5.0
bad_rat = 0.0
ingr_count = 3
calories = [2000, 2400, 2800] # len(calories) = nutr_count
mult = 2/3
nutr_count = 3
food_count = 3

### ingr_const

In [67]:
ingr_data = pd.read_csv(output_path + 'ingr_data.csv')
ingr_data.sort_values(by = 'count', ascending = False, inplace = True)
ingr_data.head()

,iid,name,count
6270,6270,salt,72848
840,840,butter,48039
2499,2499,egg,43350
5010,5010,onion,42631
6906,6906,sugar,37464


In [68]:
ingr_candidate = ingr_data.head(int(len(ingr_data)*(ingr_percent/100)))
ingr_candidate.head()

,iid,name,count
6270,6270,salt,72848
840,840,butter,48039
2499,2499,egg,43350
5010,5010,onion,42631
6906,6906,sugar,37464


In [69]:
ingr_list = ingr_candidate['iid'].tolist()

In [70]:
ingr_list[0:10]

[6270, 840, 2499, 5010, 6906, 5006, 3203, 7655, 4717, 2683]

In [71]:
recipe_data = ld.load_recipe_data()
recipe_data.head()

,name,full_id,nutrition,ingredient_ids
fid,,,,
0,white bean green chile pepper soup,40893,"[204.8, 5.0, 9.0, 26.0, 24.0, 2.0, 10.0]","[3384, 7979, 2127, 3502, 3217, 1257, 2778, 500..."
1,devilicious cookie cake delights,44394,"[132.3, 11.0, 39.0, 5.0, 4.0, 11.0, 5.0]","[912, 7557, 2499, 5382]"
2,baked potato toppings,85009,"[2786.2, 342.0, 134.0, 290.0, 161.0, 301.0, 42.0]","[4623, 6265, 1168, 6016, 3597, 3440, 7213, 169..."
3,kfc honey bbq strips,134728,"[316.0, 4.0, 40.0, 37.0, 78.0, 4.0, 10.0]","[1304, 2683, 3217, 6270, 3532, 869, 7557, 3698..."
4,lamb stew with tomatoes chickpeas and spices,200236,"[606.5, 65.0, 12.0, 34.0, 65.0, 83.0, 7.0]","[4130, 6270, 3486, 7557, 5010, 3203, 2683, 125..."


In [77]:
user_data = pd.read_csv(input_path + 'PP_users.csv')

In [78]:
user_data['items'] = user_data['items'].str.replace(" ", "")
user_data['items'] = user_data['items'].apply(lambda x: x[1:-1].split(','))
user_data['ratings'] = user_data['ratings'].str.replace(" ", "")
user_data['ratings'] = user_data['ratings'].apply(lambda x: x[1:-1].split(','))

In [79]:
user_data = user_data[user_data['u'] < 1000]
user_data.set_index('u', inplace = True)
user_data = user_data.loc[:, ['items', 'ratings']]

In [80]:
user_data

,items,ratings
u,,
0,"[1118, 27680, 32541, 137353, 16428, 28815, 658...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 4.0, ..."
1,"[122140, 77036, 156817, 76957, 68818, 155600, ...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ..."
2,"[168054, 87218, 35731, 1, 20475, 9039, 124834,...","[3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 5.0, ..."
3,"[163193, 156352, 102888, 19914, 169438, 55772,...","[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 5.0, 5.0, ..."
4,"[72857, 38652, 160427, 55772, 119999, 141777, ...","[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 4.0, 5.0, ..."
...,...,...
995,"[324, 154349]","[5.0, 5.0]"
996,"[994, 42272, 159211, 31188, 59622, 114480, 168...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ..."
997,"[324, 2949]","[5.0, 5.0]"


In [23]:
# ingredient related constraints need to be consistent
# ex) included ingredient shouldn't have bad rating
# ex) excluded ingredient shouldn't have good rating

In [81]:
ingr_column = [copy.deepcopy(ingr_list) for x in range(len(user_data))]
len(ingr_column)

1000

In [82]:
user_data = user_data.assign(include = ingr_column)
user_data = user_data.assign(exclude = copy.deepcopy(ingr_column))
user_data.head()

,items,ratings,include,exclude
u,,,,
0,"[1118, 27680, 32541, 137353, 16428, 28815, 658...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 4.0, ...","[6270, 840, 2499, 5010, 6906, 5006, 3203, 7655...","[6270, 840, 2499, 5010, 6906, 5006, 3203, 7655..."
1,"[122140, 77036, 156817, 76957, 68818, 155600, ...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[6270, 840, 2499, 5010, 6906, 5006, 3203, 7655...","[6270, 840, 2499, 5010, 6906, 5006, 3203, 7655..."
2,"[168054, 87218, 35731, 1, 20475, 9039, 124834,...","[3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 5.0, ...","[6270, 840, 2499, 5010, 6906, 5006, 3203, 7655...","[6270, 840, 2499, 5010, 6906, 5006, 3203, 7655..."
3,"[163193, 156352, 102888, 19914, 169438, 55772,...","[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 5.0, 5.0, ...","[6270, 840, 2499, 5010, 6906, 5006, 3203, 7655...","[6270, 840, 2499, 5010, 6906, 5006, 3203, 7655..."
4,"[72857, 38652, 160427, 55772, 119999, 141777, ...","[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 4.0, 5.0, ...","[6270, 840, 2499, 5010, 6906, 5006, 3203, 7655...","[6270, 840, 2499, 5010, 6906, 5006, 3203, 7655..."


In [83]:
for index, row in user_data.iterrows():
    for x in range (0, len(row['items'])):
        item_id = int(row['items'][x])
        if(item_id >= 5000): # ignore if item id is larger or equal than 5000
            continue
            
        if float(row['ratings'][x]) == good_rat : # remove ingredients included in the item frome exclude list
            ingrs = recipe_data.loc[item_id].ingredient_ids
            for ingr in ingrs:
                if int(ingr) in row['exclude']:
                    row['exclude'].remove(int(ingr))
        if float(row['ratings'][x]) == bad_rat :
            ingrs = recipe_data.loc[item_id].ingredient_ids
            for ingr in ingrs:
                if int(ingr) in row['include']:
                    row['include'].remove(int(ingr))

In [84]:
user_data.head()

,items,ratings,include,exclude
u,,,,
0,"[1118, 27680, 32541, 137353, 16428, 28815, 658...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 4.0, ...","[6270, 840, 2499, 5010, 6906, 5006, 3203, 7655...","[840, 2499, 5010, 6906, 3203, 7655, 4717, 2683..."
1,"[122140, 77036, 156817, 76957, 68818, 155600, ...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[6270, 840, 2499, 5010, 6906, 5006, 3203, 7655...","[2499, 5010, 6906, 3203, 7655, 4717, 2683, 800..."
2,"[168054, 87218, 35731, 1, 20475, 9039, 124834,...","[3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 5.0, ...","[6270, 840, 2499, 5010, 6906, 5006, 3203, 7655...","[6270, 840, 2499, 5010, 6906, 5006, 3203, 7655..."
3,"[163193, 156352, 102888, 19914, 169438, 55772,...","[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 5.0, 5.0, ...","[6270, 840, 2499, 5010, 6906, 5006, 3203, 7655...","[7449, 6654, 1257, 4987, 1124, 3723, 2856, 298..."
4,"[72857, 38652, 160427, 55772, 119999, 141777, ...","[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 4.0, 5.0, ...","[6270, 840, 2499, 5010, 6906, 5006, 7655, 4717...","[5010, 6906, 7655, 5319, 3184, 63, 6276, 4253,..."


In [85]:
def random_select(x): # pick from 0 to x
    result = []
    while len(result)<ingr_count:
        p = int(random.uniform(0, x))
        if p not in result:
            result.append(p)
    return result

In [89]:
for index, row in user_data.iterrows():
    # select random ingr to include
    if len(row['include']) > ingr_count:
        pick = random_select(len(row['include'])-1)
        result = []
        for p in pick:
            result.append(row['include'][p])
        row['include'] = result

    # select random ingr to exclude
    if len(row['exclude']) > ingr_count:
        pick = random_select(len(row['exclude'])-1)
        result = []
        for p in range (0, len(pick)):
            result.append(row['exclude'][p])
        row['exclude'] = result            

In [90]:
user_data.head()

,items,ratings,include,exclude
u,,,,
0,"[1118, 27680, 32541, 137353, 16428, 28815, 658...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 4.0, ...","[5298, 6276, 3203]","[840, 2499, 5010]"
1,"[122140, 77036, 156817, 76957, 68818, 155600, ...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[2320, 5319, 1910]","[2499, 5010, 6906]"
2,"[168054, 87218, 35731, 1, 20475, 9039, 124834,...","[3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 5.0, ...","[4717, 5185, 1329]","[6270, 840, 2499]"
3,"[163193, 156352, 102888, 19914, 169438, 55772,...","[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 5.0, 5.0, ...","[7557, 4253, 1119]","[7449, 6654, 1257]"
4,"[72857, 38652, 160427, 55772, 119999, 141777, ...","[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 4.0, 5.0, ...","[6324, 1511, 3668]","[5010, 6906, 7655]"


In [91]:
ingr_const = user_data.loc[:, ['include', 'exclude']]
ingr_const.head()

,include,exclude
u,,
0,"[5298, 6276, 3203]","[840, 2499, 5010]"
1,"[2320, 5319, 1910]","[2499, 5010, 6906]"
2,"[4717, 5185, 1329]","[6270, 840, 2499]"
3,"[7557, 4253, 1119]","[7449, 6654, 1257]"
4,"[6324, 1511, 3668]","[5010, 6906, 7655]"


In [92]:
ingr_const.to_csv('../data/ingr_const.csv')

## Nutrition related Constraints

In [5]:
data = {
        'calories': [x * mult for x in  calories],
        'total fat': [100 * mult] * nutr_count,
        'sugar': [100 * mult] * nutr_count,
        'sodium': [100 * mult] * nutr_count,
        'protein': [100 * mult] * nutr_count,
        'saturated fat': [100 * mult] * nutr_count,
        'carbohydrates' : [100 * mult] * nutr_count    
       }

In [6]:
nutr_sample = pd.DataFrame(data)
nutr_sample.head()

,calories,total fat,sugar,sodium,protein,saturated fat,carbohydrates
0,1333.333333,66.666667,66.666667,66.666667,66.666667,66.666667,66.666667
1,1600.000000,66.666667,66.666667,66.666667,66.666667,66.666667,66.666667
2,1866.666667,66.666667,66.666667,66.666667,66.666667,66.666667,66.666667


In [8]:
nutr_sample.to_csv('../data/nutr_sample.csv', index = False)

In [98]:
appear = [0] * 5000

In [100]:
for index, row in user_data.iterrows():
    for x in range (0, len(row['items'])):
        item_id = int(row['items'][x])
        if item_id < 5000:
            appear[item_id] = appear[item_id] + 1

In [102]:
food_count_data = pd.DataFrame(appear)
food_count_data.sort_values(by = 0, ascending = False, inplace = True)
food_count_data

,0
267,82
279,60
156,57
994,50
1984,49
...,...
1528,0
3515,0
3513,0
1533,0


In [104]:
food_list = food_count_data.head(food_count).index.values.tolist()
food_list

[267, 279, 156]

In [105]:
food_list_data = pd.DataFrame(food_list)
food_list_data

,0
0,267
1,279
2,156


In [106]:
food_list_data.to_csv('../data/food_const.csv', index = False)